In [ ]:
from osgeo import gdal
import os
import numpy as np
import cv2
import re
import matplotlib.pyplot as plt
import sentinel1decoder

In [ ]:
def start_points(size, split_size, overlap=0): # overlap = % of overlap between tiles, both rows and columns 
    points = [0]
    stride = int(split_size * (1-overlap))
    counter = 1
    while True:
        pt = stride * counter
        if pt + split_size >= size:
            points.append(size - split_size)
            break
        else:
            points.append(pt)
        counter += 1
    return points


def create_patches_labels(L1, folder, i):     
    patch_dir = os.path.join(folder, 'L1_tile')
    if not os.path.isdir(patch_dir):
        os.mkdir(patch_dir)
        
    img_h, img_w = L1.shape
    split_width = 3034 # define tile size 
    split_height = 916

    X_points = start_points(img_w, split_width, 0)
    Y_points = start_points(img_h, split_height, 0)
    name = 'splitted'
    
    tot = []
    for s in Y_points:
        for j in X_points:
            split = L1[s:s+split_height, j:j+split_width]
            patch_name = f'patch_{name}_{i}_{s:02}_{j:02}.npy'
            split = split.astype(np.float32)
            np.save(os.path.join(patch_dir, patch_name), split)
            tot.append(split)        
    return tot



def create_patches(re, im, folder, i):      
    patch_dir = os.path.join(folder, 'L0_tile')
    if not os.path.isdir(patch_dir):
        os.mkdir(patch_dir)
        
    img_h, img_w = re.shape #or im
    split_width = 3034
    split_height = 916

    X_points = start_points(img_w, split_width, 0)
    Y_points = start_points(img_h, split_height, 0)
    name = 'splitted'
    
    tot = []
    for s in Y_points:
        for j in X_points:
            split_re = re[s:s+split_height, j:j+split_width]

            split_im = im[s:s+split_height, j:j+split_width]
            
            split = cv2.merge([split_re, split_im])
            split = split.astype(np.float32)
            
            patch_name = f'patch_{name}_{i}_{s:02}_{j:02}.npy'
            np.save(os.path.join(patch_dir, patch_name), split)
            tot.append(split)
        
    return tot

In [ ]:
path = "L0"
L0_files =[]
for root, dirs, files in os.walk("L0/"):
    for file in files:
        strin1 = 'annot'
        string2 = 'index'
        if file.endswith(".dat") and strin1 not in file and string2 not in file:
            L0_files.append(root + '/' + file)
print(L0_files)

path = "L1"
L1_files = []
for root, dirs, files in os.walk(path):
    for file in files:
        check = str(os.path.join(root, file))
        if(file.endswith("001.tiff")):
            L1_files.append(os.path.join(root,file))
print(L1_files)

In [ ]:
path = "L1"
i = 0
L1_size = []
for root, dirs, files in os.walk(path):
    for file in files:
        check = str(os.path.join(root, file))
        if(file.endswith("001.tiff")):

            print("Current file: ", os.path.join(root, file))
            #Load L1 to obtain same dimension for L0 and L1
            raster = gdal.Open(os.path.join(root,file))

            rasterArray = raster.ReadAsArray()
            print('SLC data type: ', rasterArray.dtype)
            rasterArray = rasterArray.reshape(len(rasterArray),len(rasterArray[0]))
            print('SLC shape: ', rasterArray.shape)

            image_SLC = abs(rasterArray)
            print('SLC abs data type: ',image_SLC.dtype)

            del rasterArray
            #Delate L1 raws and columns to have azimuth and range proportional to 916 and 3034
            Range = 916
            Azimuth = 3034
            num_row = len(image_SLC)//Range
            num_col = len(image_SLC[0])//Azimuth
            print("New number of row: ", num_row*Range)
            print("New number of columns: ", num_col*Azimuth)
            SLC_r = len(image_SLC)
            SLC_c = len(image_SLC[0])
            r = SLC_r - num_row*Range #to be subtracted
            c = SLC_c - num_col*Azimuth
            L1 = image_SLC[0:SLC_r-r, 0:SLC_c-c]
            L1_size.append(L1.shape)
            print("New L1 dimension: ", L1.shape)

            del image_SLC
            #plt.figure(figsize = (20,20))
            #plt.imshow(L1[0:916,0:3034], cmap = 'gray', vmin = 0, vmax = 255)
            #plt.colorbar()

            folder = ''
            L1_patches = create_patches_labels(L1, folder, i)
            print('Tot number of patches for the current image: ', len(L1_patches))
            i += 1
            print("Dim of a single patch: ", L1_patches[len(L1_patches)-1].shape)
            del L1_patches, L1

print(L1_size)

In [ ]:
path = 'L0'
i = 0 # per la size di L1 corrispondente
j = 0
path = "L0"
for file in L0_files[0:len(L0_files):2]:

    print("Current file: ", file)
    l0file = sentinel1decoder.Level0File(file)
    # print(l0file.packet_metadata)
    # print(l0file.ephemeris)

    selected_burst = 9
    selection = l0file.get_burst_metadata(selected_burst)
    print(selection)
    # Decode the IQ data
    radar_data = l0file.get_burst_data(selected_burst)

    plt.figure(figsize=(12, 12))
    plt.title("Sentinel-1 Raw I/Q Sensor Output")
    plt.imshow(abs(radar_data[:200,:200]), vmin=0, vmax=15, origin='lower')
    plt.xlabel("Fast Time (down range)")
    plt.ylabel("Slow Time (cross range)")
    plt.show()

    raw_L1, col_L1 = L1_size[i]
    raw_L0, col_L0 = radar_data.shape
    print("RAW data size: ", radar_data.shape)
    print("SLC data size: ", L1_size[i])
    raw = raw_L0 - raw_L1
    col = col_L0 - col_L1
    print("Raw and Columns to be subtraced: ", raw, col)
    L0 = radar_data[0:raw_L0-raw,0:col_L0-col]
    print("Check final L0 shape: ", L0.shape)
    
    del radar_data, raw, col
    #plt.figure(figsize = (20,20))
    #plt.imshow(L0_re[0:917,0:3034], cmap = 'gray', vmin = L0_re.min(), vmax = L0_re.max())
    #plt.colorbar()
    #plt.close()
    
    #Cut patches
    folder = ''
    L0_patches = create_patches(L0.real, L0.imag, folder, j)
    print('Tot number of patches for the current image: ', len(L0_patches))
    print("Dim of a single patch: ", L0_patches[len(L0_patches)-1].shape)
    i = i + 1
    j = j + 1
    print(i)
    del L0, L0_patches
